# 6 Comprehensions, Generators and Lazy Evaluation
##### **Author: Adam Gatt**

## Comprehensions

Python allows for the explicit definition of lists, sets and dicts using "comprehensions". Rather than an explicit listing of the items in the collection, it is instead a formula of how to construct the collection.

### List Comprehensions
These take the form of:
```
[<expression> for <item> in <iterable>]
```
You can filter whether items should be processed with:
```
[<expression> for <item> in <iterable> if <condition>]
```
You can break the elements into individual lines for readability, e.g.



In [ ]:
odd_squares = [
  x**2
  for x in range(10)
  if x%2 == 1
]

print(odd_squares)

[1, 9, 25, 49, 81]


What if you don't want your condition to filter out items, but instead just treat them differently? Then you can work the condition into the expression computation, e.g.:
```
[<expression_1> if <condition> else <expression_2> for <item> in <iterable>]
```

In [ ]:
odds_should_be_negative = [
  (x if x%2==0 else -x)
  for x in range(10)
]

print(odds_should_be_negative)

[0, -1, 2, -3, 4, -5, 6, -7, 8, -9]


This works because you can use any expression in the comprehension. If you have some complex handling to perform then it's probably better to offload the computation to a separate function than to have a big ternary operator chain ruining the readability of your comprehension.

In [ ]:
def fizzbuzz(x):
  return ('fizzbuzz' if x%3==0 and x%5==0
  else 'fizz' if x%3==0
  else 'buzz' if x%5==0
  else str(x))

print(', '.join([fizzbuzz(x) for x in range(1, 20)]))

1, 2, fizz, 4, buzz, fizz, 7, 8, fizz, buzz, 11, fizz, 13, 14, fizzbuzz, 16, 17, fizz, 19


### Set Comprehensions
These work the same as list comprehensions but the results will be cast as a set. The most important property of this is that the result will not contain duplicates.

In [ ]:
governments_by_country = {
    'France': 'Republic',
    'Mexico': 'Republic',
    'Australia': 'Constitutional Monarchy',
    'Brunei': 'Absolute Monarchy',
    'Norway': 'Constitutional Monarchy',
    'Saudi Arabia': 'Absolute Monarchy'
}

# What sorts of governments do we know about? Calling values() will have duplicates
print(governments_by_country.values())

# A set comprehension will not include duplicates in its response
print()
print({government for government in governments_by_country.values()})

# With no special processing or filtering, the above is equivalent to
# "set(government_by_country.values())"

# I only want to know about types of monarchies
print()
print({
    government
    for government in governments_by_country.values()
    if 'Monarchy' in government
})

dict_values(['Republic', 'Republic', 'Constitutional Monarchy', 'Absolute Monarchy', 'Constitutional Monarchy', 'Absolute Monarchy'])

{'Republic', 'Absolute Monarchy', 'Constitutional Monarchy'}

{'Absolute Monarchy', 'Constitutional Monarchy'}


### Dict Comprehensions
You can even build a dict with a comprehension. Just like how dicts and sets both use curly braces, dict and set comprehensions both use curly braces as well. A dict comprehension is indicated by the `key: value` format of its expression.

In [ ]:
# Example: Inverting a dictionary to swap keys and values
class_by_lesson = {
    'Magnetism': 'Physics',
    'Osmosis': 'Biology',
    'Motion': 'Physics',
    'Gravity': 'Physics',
    'Oxidisation': 'Chemistry',
    'Catalysts': 'Chemistry'
}

# Unique values will be the keys of our inverted dictionary
unique_classes = set(class_by_lesson.values())

# Dict comprehension has an expression format of <key>: <value>
# Both the key and the value can be expressions that evaluate to something
# Here the key is forced to upper case, and the value is itself a comprehension (list)
lessons_by_class = {
    class_name.upper(): [
      lesson
      for lesson in class_by_lesson.keys()
      if class_by_lesson[lesson]==class_name
    ]
    for class_name in unique_classes
}

print(lessons_by_class)

{'CHEMISTRY': ['Oxidisation', 'Catalysts'], 'PHYSICS': ['Magnetism', 'Motion', 'Gravity'], 'BIOLOGY': ['Osmosis']}


### Tuple Comprehensions
Tuples are iterables and have their own brackets `()`, so can we make a tuple comprehension? Yes, but unfortunately not in the way we would expect. We might expect the same format as a list comprehension but with parenthesis `()` instead of square brackets, but actually, this phrasing is already in use for the creation of "generators" (see later in this notebook).

Instead, we have to create a list comprehension and then call the `tuple()` function to convert it.

In [ ]:
# Can we use the expected syntax to make a tuple comprehension?
student_scores = (60, 25, 87, 89, 54, 66)
passing_scores = (score for score in student_scores if score > 65)

# No! We've created a generator, whatever that is.
print(passing_scores)

# We need to use a list comprehension and convert to a tuple
passing_scores = tuple([score for score in student_scores if score > 65])
print()
print(passing_scores)

# Actually, we can leave out the square brackets entirely too. How is this
# allowed? Well, we are creating what's called a "generator expression" and
# using it as the argument to the tuple() function. Generators will be
# covered later in this notebook.
passing_scores = tuple(score for score in student_scores if score > 65)
print()
print(passing_scores)

<generator object <genexpr> at 0x7faf4d2a42b0>

(87, 89, 66)

(87, 89, 66)


## Higher-level comprehensions

Comprehensions can often be used to replace a simple for-loop, with some filtering and mapping of results thrown in if needed. Can we also use a comprehension to simulate a nested for-loop (i.e. a for-loop within another for-loop)? 

Yes! We can do this with a second-level comprehension. This takes the format of:
```
[<expression> for <inner iterable> in <outer iterable> for <item> in <inner iterable>]
```
Or, perhaps more readably:
```
[
  <expression>
  for <inner iterable> in <outer iterable>
  for <item> in <inner iterable>
]
```
This will have the effect of "flattening" out the nested structure of our outer iterable.

In [ ]:
pet_names = [['Rex', 'Mittens'], ['Fido'], ['Felix', 'Garfield', 'Sylvester']]

print([
  f"Treat for {name}"
  for inner_list in pet_names
  for name in inner_list
])

['Treat for Rex', 'Treat for Mittens', 'Treat for Fido', 'Treat for Felix', 'Treat for Garfield', 'Treat for Sylvester']


This phrasing is a little unintuitive. No doubt the readability would be improved if the "for" clauses were in the opposite order. You need to just know the understanding that you begin with the outer-most iteration and then proceed inwards. With this approach, you can have a third- or even higher-level comprehension as high as you want.
```
[
  <expression>
  for <iterable n-1> in <iterable n>
  for <iterable n-2> in <iterable n-1>
  ...
  for <iterable 1> in <iterable 2>
  for <item> in <iterable 1>
]
```

In [ ]:
# An example with dicts and sets

flag_colours_by_country = {
    'Australia': {'Red', 'Blue', 'White'},
    'France': {'Blue', 'Red', 'White'},
    'Norway': {'White', 'Red', 'Blue'},
    'Saudi Arabia': {'Green', 'White'}
}

# What are all the colours present in these flags?
known_colours = {
  colour # Expression
  for flag_colours in flag_colours_by_country.values() # Outer loop
  for colour in flag_colours # Inner loop
}

print()
print(known_colours)



{'Blue', 'Red', 'White', 'Green'}


Can't we achieve this by iterating through each set of colours and unpacking them all into the same comprehension? Unfortunately Python doesn't allow unpacking into a comprehension, and PEPs have been considered on this issue but have been considered as unacceptably ambiguous to parse.

In [ ]:
print({*colours for colours in flag_colours_by_country.values()})

SyntaxError: ignored

Yet we can achieve this if we unpack them into the definition of a regular "set literal"

In [ ]:
known_colours = {
  *flag_colours_by_country['Australia'],
  *flag_colours_by_country['France'],
  *flag_colours_by_country['Norway'],
  *flag_colours_by_country['Saudi Arabia']
}

print(known_colours)

{'Blue', 'White', 'Red', 'Green'}


## Generators

A generator is a way to compute items in a sequence, similar to list comprehensions. But generators are different in that they only compute each value at the moment when it is called upon.

Whereas a list comprehension will compute the entire sequence upon creation, a generator is created without computing anything. Instead, when the next value is requested from the generator, that value is computed right at that moment and provided. This is known as **Lazy Evaluation**.

Because of this behaviour:

* Generators never waste time computing elements that are not used
* Generators can provide each element as soon as they are available, instead of waiting for the entire sequence to be computed before any elements are provided
* A generator may provide items indefinitely, effectively representing an infinite sequence.
* Generators are not required to have a known or fixed length before processing has begun.
* Generators are only required to proceed through their sequence of items once. If a generator runs out of items and then is iterated through again, it is __exhausted__ and will provide no new items.

### Generator Expressions
One way to create a generator is through a "generator expression". These look the same as a list comprehension but with round brackets `()`.

*Note: Looking like comprehensions you may be tempted to call them "Generator Comprehensions", but know that their proper name is "Generator Expressions".*

In [ ]:
from time import sleep

def slow_square(x):
  sleep(2)
  return x**2

In [ ]:
# List comprehension is evaluated once its defined
as_list = [slow_square(x) for x in range(5)]
print(as_list)

[0, 1, 4, 9, 16]


In [ ]:
# Once the list is created we can iterate quickly
for square in as_list:
  print(square)

0
1
4
9
16


In [ ]:
# The generator expression doesn't evaluate any values upon definition, creation is very quick
as_generator = (slow_square(x) for x in range(5))
print(as_generator)

<generator object <genexpr> at 0x7f624e7b2d58>


In [ ]:
# But the values are evaluated when fetched from the generator
for square in as_generator:
  print(square)

0
1
4
9
16


#### As arguments to function calls
Generator expressions are identified by the brackets `()` surrounding them. Function calls also require the use of these round brackets. You might think that supplying a generator expression to a function would result in these brackets appearing doubled up:

`my_function((<generator expression>))`

But Python provides a little shortcut allowing you to use only a single set of brackets, which has a slightly neater appearance in my opinion.

In [ ]:
fibonacci = [1, 1, 2, 3, 5, 8, 13, 21, 34]

# Generator expression inside of a function call results in doubled brackets
sum_of_odd_fibonacci = sum((x for x in fibonacci if x%2==1))
print(sum_of_odd_fibonacci)

# Python allows us to use a single set as a shortcut
sum_of_odd_fibonacci = sum(x for x in fibonacci if x%2==1)
print()
print(sum_of_odd_fibonacci)

44

44


In [ ]:
' * '.join(name for name in governments_by_country.keys() if len(name) <=6)

'France * Mexico * Brunei * Norway'

### Generator Function
The second way to define a generator is with a "generator function". These are essentially created like regular functions but instead of using `return` for the intended output, you use the `yield` keyword instead. The output from calling the function is a generator object.

In these functions, `yield` isn't the last line, like "return" often is. When the created generator is iterated over, execution will pause at the `yield` statement to provide a value and then will pick up from that same place when execution resumes.

Generator functions have an advantage over generator expressions in that we can preserve some state between iterations. There will be an example of this later.

In [ ]:
# "yield" instead of "return" = generator function
def squares_less_than(upper_bound):
  x = 0
  while x**2 < upper_bound:
    yield x**2
    x += 1

# "squares_less_than" is a function, like all functions
print(squares_less_than)

# When we call it, there is no evaluated result. Instead it
# returns a generator that we can then ask for values by
# iterating over it
my_output = squares_less_than(30)
print(my_output)

<function squares_less_than at 0x7f624e7bc6a8>
<generator object squares_less_than at 0x7f624e7b2bf8>


In [ ]:
# How do I get the generator to provide its computed values? We
# can use it with anything that accepts an iterable. One example
# is list(), which creates a list out of the submitted iterable.
print(list(my_output))

# sum() also operates on iterables and so will cause evaluation
print(sum(squares_less_than(30)))

[0, 1, 4, 9, 16, 25]
55


In [ ]:
# And of course we can run a for-loop on an iterable
for square in squares_less_than(20):
  print(f"{square} is less than 20")

0 is less than 20
1 is less than 20
4 is less than 20
9 is less than 20
16 is less than 20


In [ ]:
# Generator expressions can also be used on functions that
# accept iterables
sums_1 = sum((x**2 for x in range(10)))
print(sums_1)

# As a shortcut, one set of brackets can actually be omitted,
# allowing us to write the generator directly into the function
sums_2 = sum(x**2 for x in range(10))
print(sums_2)

285
285


In [ ]:
animals = ['cat', 'horse', 'elephant', 'mouse']
max(len(word) for word in animals)

8

### Use: Avoid need to evaluate entire list

In [ ]:
def fibonacci(x):
  return 1 if x <= 2 else fibonacci(x-1)+fibonacci(x-2)

# This list comprehension will compute the entire list
# before even the first iteration occurs
for x in [fibonacci(x) for x in range(1, 40)]:
  print(x)
  if x > 100:
    print("That's large enough")
    break

1
1
2
3
5
8
13
21
34
55
89
144
That's large enough


In [ ]:
# This generator expression will only compute each value before
# its iteration. The very costly calculations near the end of the
# range are never computed as the loop is exited before we reach them
for x in (fibonacci(x) for x in range(1, 40)):
  print(x)
  if x > 100:
    print("That's large enough")
    break

1
1
2
3
5
8
13
21
34
55
89
144
That's large enough


### Use: Infinite processing
If an generator function does not have a condition that will cause it to exhaust, then it will continue to iterate forever. A generator expression might also iterate forever if it is defined from a source that is infinite.Examples can be found in the `itertools` module, which provides a number of functions and iterators for efficient looping. The difference between generators and iterators can be subtle and we will touch on it later.

Examples of infinite iterators are count(), cycle() and repeat(). If you attempt to cast them to a list or use in a comprehension then they will continue returning items forever and the operation will not finish. Likewise, a generator computed from these sources will continue forever under those operations.

In [ ]:
from itertools import count, islice

# count() is an infinite source of natural numbers, but we can
# safely use it in a generator
evens = (x for x in count(1) if x%2 == 0)

# islice() allows us to take a slice of any iterator, here we
# use it to fetch the first 10 items from our "evens" generator
list(islice(evens, 10))

[2, 4, 6, 8, 10, 12, 14, 16, 18, 20]

In [ ]:
# What happens if "evens" used list comprehension instead of
# a generator expression?
evens = [x for x in count(1) if x%2 == 0]

# It will calculate forever and never reach this statement
evens[:10]

KeyboardInterrupt: ignored

### Use: Keep state in between computed values

In [ ]:
from itertools import islice

def fibonacci_generator():
  # Initialise first two fibonacci numbers
  x_minus_2, x_minus_1 = 1, 1
  # Return these first two numbers explicitly, later values will be calculated
  yield x_minus_2
  yield x_minus_1
  while True:
    # New number is sum of previous two numbers
    x = x_minus_2 + x_minus_1
    yield x
    # Update our remembered previous numbers
    x_minus_2, x_minus_1 = x_minus_1, x


for number in islice(fibonacci_generator(), 8):
  print(number)

1
1
2
3
5
8
13
21


In [ ]:
list(islice(fibonacci_generator(), 8))

[1, 1, 2, 3, 5, 8, 13, 21]

### Manual iteration with `next()`
We can call `next()` with a generator to compute and fetch the next element manually.

In [ ]:
fibgen = fibonacci_generator()
print(next(fibgen))
print(next(fibgen))
print(next(fibgen))
print(next(fibgen))
_ = next(fibgen) # This value won't be printed, just discarded
print(next(fibgen))

1
1
2
3
8


### Exhaustion
As mentioned earlier, a generator that runs out of items will become exhausted. This is indicated by throwing a `StopIteration` exception. Operations that process through iterables, such as list comprehensions and for-loops, will look for and handle this exception instead of displaying it to the console.

However, once a generator is exhausted it does not reset to the start of the sequence. It will continue providing `StopIteration` when asked for another item. Using the generator in a loop again will result in zero iterations. Using it in a comprehension or list/set/tuple cast will result in an output of zero length.

In [ ]:
# Create a generator that creates statements for each city in a list
itinerary = (f"Stop {idx+1}: {city}" for idx, city in enumerate(['San Diego', 'Los Angeles', 'San Francisco', 'Alameda']))

print(itinerary) # Print the generator

# Let's evaluate the generator into a list with the list() function
print()
print(list(itinerary))

# Attempting this again won't work because the generator has already been
# processed once. We will need to re-create the generator if we want to iterate through it again
print()
print(list(itinerary))

<generator object <genexpr> at 0x7fc7f701f150>

['Stop 1: San Diego', 'Stop 2: Los Angeles', 'Stop 3: San Francisco', 'Stop 4: Alameda']

['Stop 1: San Diego', 'Stop 2: Los Angeles', 'Stop 3: San Francisco', 'Stop 4: Alameda']


In [ ]:
# Let's create the generator again and step through it manually
itinerary = (f"Stop {idx+1}: {city}" for idx, city in enumerate(['San Diego', 'Los Angeles', 'San Francisco', 'Alameda']))

print(next(itinerary))
print(next(itinerary))
print(next(itinerary))
print(next(itinerary))
print(next(itinerary)) # We've run out of items and StopIteration will be thrown

Stop 1: San Diego
Stop 2: Los Angeles
Stop 3: San Francisco
Stop 4: Alameda


StopIteration: ignored

In [ ]:
# Let's step through the first element and then evaluate the generator
itinerary = (f"Stop {idx+1}: {city}" for idx, city in enumerate(['San Diego', 'Los Angeles', 'San Francisco', 'Alameda']))

print(next(itinerary))
print(list(itinerary))

# Not all values were provided to list() because the generator already provided
# the first value when it was called with next(). Calling it with next() now
# will throw a StopIteration, but using it in a comprehension or loop will
# handle this gracefully
for remaining_city in itinerary:
  print('Did we forget to visit {}?'.format(remaining_city))

Stop 1: San Diego
['Stop 2: Los Angeles', 'Stop 3: San Francisco', 'Stop 4: Alameda']


### Usage Considerations
* Because generators can only be iterated over once, it is best practice to use them as soon as possible after their creation. You might not even need to create them as a variable, but instead use the generator expression or generator function call in-line with the code that consumes it. A generator that is created and then sits unused for a long time has a greater chance of accidentally being iterated over twice as its disposable nature becomes forgotten by the coder.

* If you are using a list comprehension for the sole purpose of immediately iterating over the resulting list (*and only once!*), then consider replacing it with a generator expression instead. This will serve the same purpose but will bring the incidental advantages of lazy evaluation.

## Lazy evaluation
In general programming, Lazy Evaluation is an approach that comes in useful when you need to process a sequence of elements, and that processing involves multiple expensive steps. Traditional imperative programming conventionally uses "eager evaluation", where each step will process the entire sequence, storing the intermediate results in new variables. This comes with some costs:

*   Intermediate variables must be large enough to hold the results for the entire sequence.
*   No results are available until the final step is processed, i.e. all operations have been performed on the entire sequence.


In contrast, lazy evaluation is an approach where we define how the calculations should be performed, without performing them just yet. Instead we construct a model of these step processes, and at the end the entire model is evaluated on the input data all at once. When the input is a sequence, we can apply the entire model to each element in turn, similar to stream processing. This means that:

* Intermediate variables only need enough memory to hold results for a single element at a time
* The results from each element are made available as soon as they are computed (allowing for immediate display, writing to file, used for HTTP request, etc)
* If there is no interaction or memory between elements in the sequence then this may make our processing easy to parallelise

### Example 1: Processing despite blocking input
The `input()` function is a blocking operation. It halts processing entirely until input is provided from "system in" (in this case, the keyboard). `name_stream` is a generator that will provide four names, but only provide each when requested. If it were a list comprehension, we would need to enter all four names before any further processing could proceed. By using generators we can perform later operations on provided values before requesting the next name and blocking again.

In [ ]:
# This generator:
# 1) reads a name from the keyboard
name_stream = (input('\nWhat is your name? ') for _ in range(4))

# This generator reads names from an input, writes them to a database,
# and then passes them on.
def record_user(names_in):
  for name in names_in:
    print('Recording name to database')
    yield name

# This generator:
# 1) reads a name from the keyboard
# 2) records to database
record_users_when_submitted = record_user(name_stream)

# This generator:
# 1) reads a name from the keyboard
# 2) records to database
# 3) transforms the name into a greeting
greetings = (f"Hello {name}" for name in record_users_when_submitted)

# We have set up a number of generators but absolutely nothing has been
# processed or computed yet. Generators only perform computation when
# an element is requested.

# By iterating through the final generator (with the for-loop), we ask it
# to emit elements and so actual computation finally begins.
for greeting in greetings:
  print(f"Greeting message is: {greeting}")

### Example 2: Processing despite infinite input
This time we use count() to request an infinite number of keyboard inputs, and so waiting for them to complete means that we will wait forever and no further operations are possible. By using generators we hold off on evaluating anything until values are requested. This means we can  

In [ ]:
from itertools import count

# This generator will now provide infinite names. We definitely do
# not want this to be an eager-evaluated list comprehension now!
name_stream = (input('\nWhat is your name? ') for _ in count())

# Other generators chained together as previously. Again, nothing is
# computed at all until an element is requested from a generator.
record_users_when_submitted = record_user(name_stream)
greetings = (f"Hello {name}" for name in record_users_when_submitted)

# We are safe to iterate through the infinite generator if we have
# some condition for breaking out of it, or at the least limit it
# to a particular number of iterations with "islice()""
for greeting in greetings:
  if 'goodbye' in greeting:
    break
  else:
    print(f"Greeting message is: {greeting}")


What is your name? Adam
Recording name to database
Greeting message is: Hello Adam

What is your name? Sebastien
Recording name to database
Greeting message is: Hello Sebastien

What is your name? goodbye
Recording name to database


### Dummy example: Training an ML algorithm
This is a longer example that uses second-level generators, generators that open and close multiple files in sequence, and generators that require multiple elements from another. This is a demonstrative example - the data is faked and no model is actually trained, but it exemplifies the order of execution as each data point is submitted through the processing stream. 

In [ ]:
### DUMMY OBJECTS FOR THE EXAMPLE ###

# Context Manager that returns dummy data from pretend files
class FakeFileReader(object):
  fake_contents = {
      'original_training_set.txt': ['A' + str(x) for x in range(30)],
      'training_data_v1.0.txt': ['B' + str(x) for x in range(70)],
      'auxillary_data.txt': ['C' + str(x) for x in range(40)],
      'largest_data_set.txt': ['D' + str(x) for x in range(20)]
  }

  def __init__(self, file_name):
    self.file_name = file_name

  def __enter__(self):
    return self
  
  def __exit__(self, type, value, traceback):
    pass
  
  def read_lines(self):
    return self.fake_contents[self.file_name]

# Creates a FakeFileReader with the specified file name
def fake_open(file_name, read_write_mode):
  return FakeFileReader(file_name)

# Dummy neural network model that exposes a dummy "train" method, and
# dummy "weights" that increase every time it is trained
class FakeNeuralNet(object):
  def __init__(self):
    self.weights = [0] * 5

  def train(self, data_point, verbose=False):
    self.weights = [weight + 0.1 for weight in self.weights]
    if verbose:
      print(f'I am being trained on data point {data_point}')

  def get_weights(self):
    return self.weights;

# Create an instance of it ready to be used
my_neural_net = FakeNeuralNet()

In [ ]:
### SETTING UP OUR STREAM PROCESSING ###

# We have a list of file paths for our training data
training_data_files = ['original_training_set.txt', 'training_data_v1.0.txt', 'auxillary_data.txt', 'largest_data_set.txt']


# We create a generator function that will create a generator that, when iterated,
# will provide the datapoints from a file, one-by-one 
def file_contents(file_path):
  # Context manager will safely close when the function ends (it will leave scope)
  with fake_open(file_path, 'r') as f:
    for data_point in f.read_lines():
      yield data_point

# A generator to provide each subsequent data point from each subsequent file. We
# use a two-level structure to flatten all the data into a single sequence. Note
# how file_contents will be called four times, once for each training data file.
# As that function uses a context manager, each file will be elegantly opened
# and closed in turn as its contents become exhausted.
data_set = (data_point
            for file_name in training_data_files
            for data_point in file_contents(file_name))

# A generator function that trains a model on data points and emits the model's
# trained weights after a number of iterations
def train_model(model, training_data):
  for idx, data_point in enumerate(training_data):
    model.train(data_point, verbose=True)

    # Emit model weights every 50 iterations as a checkpoint
    if idx % 50 == 0:
      print('Saving checkpoint')
      yield model.get_weights()

  # Final report of model weights when we have run out of data
  yield model.get_weights()

# Calling the function is what creates our actual generator, ready to
# be iterated over
my_model_training_process = train_model(my_neural_net, data_set)

What would happen when we evaluate `next(my_model_training_process)` and kick off the whole process?
1. The next() statement will run `my_model_training_process` until it emits its first value.
2. We can see in the `train_model` function that this will happen after the first 50 datapoints have been evaluated.
3. The datapoints are fetched from the submitted training data, which we specified as the `data_set` generator. This will undergo 50 iterations.
4. data_set will return the first 50 elements, calling `file_contents(file)` for each file and iterating through the data points returned within.
5. `file_contents()` is a generator for each file that will ensure the files are cleanly closed after they have been used. Only one file is ever open at a time.

In [ ]:
### EXECUTING THE LAZY EVALUATION STREAM ###

# We have created a lot of generators but nothing has been computed until this
# moment, when we take the outer-most generator (my_model_training_process) and
# request an element from it. We will do this manually with next()
weights_at_first_checkpoint = next(my_model_training_process)
print('Weights at first checkpoint:')
print(weights_at_first_checkpoint)

# An alternative is to get all the weights by submitting the generator to list()
weights_over_time = list(my_model_training_process)
print('Weights at each checkpoint:')
print('\n'.join(str(weights) for weights in weights_over_time))

I am being trained on data point A0
Saving checkpoint
Weights at first checkpoint:
[16.09999999999996, 16.09999999999996, 16.09999999999996, 16.09999999999996, 16.09999999999996]
I am being trained on data point A1
I am being trained on data point A2
I am being trained on data point A3
I am being trained on data point A4
I am being trained on data point A5
I am being trained on data point A6
I am being trained on data point A7
I am being trained on data point A8
I am being trained on data point A9
I am being trained on data point A10
I am being trained on data point A11
I am being trained on data point A12
I am being trained on data point A13
I am being trained on data point A14
I am being trained on data point A15
I am being trained on data point A16
I am being trained on data point A17
I am being trained on data point A18
I am being trained on data point A19
I am being trained on data point A20
I am being trained on data point A21
I am being trained on data point A22
I am being train